# Implémentation complète en Apache Spark de l'algorithme proposé à l'aide de PySpark

In [3]:
import numpy as np
import pandas as pd
import time
import itertools
#import os
#import shutil
#import sys
#import argparse

from scipy.sparse import csr_matrix as csr


# Selon la configuration de Spark, les lignes suivantes peuvent être nécessaires.
!pip install pyspark
import pyspark as spark
from pyspark.sql import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=43de07ec7f43273d8de3ff2a94c1407a720adfb1e54e0b8074caf1afdc4983ed
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


## Fonctions requises pour calculer le coefficient de corrélation de rang flou.

In [4]:
# Norme T de Lukasiewicz.
def T_norm(x,y):
    return max(0,x+y-1)

# TL-E-Commande fortement complète dans R.  
def Fuzzy_ordering(x,y,r):
    return min(1,max(0,1-((x-y)/r)))

# Relation floue stricte (Rx ou Ry)
def Fuzzy_relation(x1,x2,r):
    return 1-Fuzzy_ordering(x2,x1,r)

# Le degré d'accord entre deux paires étant donné un r sera alors calculé comme suit.
def Concordance_degree(pair1,pair2,r):
    return T_norm(Fuzzy_relation(pair1[0], pair1[1],r), Fuzzy_relation(pair2[0], pair2[1],r))


## Génération de la matrice d'adjacence
Cette fonction renvoie la matrice des degrés d'accord de l'ensemble des motifs de taille k = 2 qui lui est transmise en tant qu'argument, ainsi qu'un r et l'ensemble de données.

In [5]:
# Il utilise un flotteur 16 bits au lieu du flotteur python standard qui utilise 64 bits.
def generate_matrix(candidate, df, r):
    
    at1 = pd.to_numeric(df.value[candidate[0][0]])
    op1 = candidate[0][1]
    
    at2 = pd.to_numeric(df.value[candidate[1][0]])
    op2 = candidate[1][1]
    
    N = len(at1)     # N est le nombre d'attributs
    matrix = np.zeros((N, N), dtype = np.float16)
    
    if op1 == '>' and op2 == '>': 
        for i in range(N):
            for j in range(N):
                if i != j:
                    matrix[i,j] = Concordance_degree([at1[i], at1[j]],[at2[i], at2[j]],r)
                    
    if op1 == '>' and op2 == '<': 
        for i in range(N):
            for j in range(N):
                if i != j:
                    matrix[i,j] = Concordance_degree([at1[i], at1[j]],[at2[j], at2[i]],r)
                    
    return (tuple(candidate), csr(matrix))
                
"""
    Nous évaluerons ces autres combinaisons puisqu'il n'y aura que des itemsets de taille 2 du
    type (('atributo1', '>'), ('atributo2', '>')) et (('atributo1', '>'), ('atributo2', '<'))

    if op1 == '<' and op2 == '>': 
        for i in range(N):
            for j in range(N):
                if i != j:
                    matrix[i,j] = Concordance_degree([at1[j], at1[i]],[at2[i], at2[j]],r)          
    
    

    if op1 == '<' and op2 == '<': 
        for i in range(N):
            for j in range(N):
                if i != j:
                    matrix[i,j] = Concordance_degree([at1[j], at1[i]],[at2[j], at2[i]],r)
"""                 
    


                    


"\n    Nous évaluerons ces autres combinaisons puisqu'il n'y aura que des itemsets de taille 2 du\n    type (('atributo1', '>'), ('atributo2', '>')) et (('atributo1', '>'), ('atributo2', '<'))\n\n    if op1 == '<' and op2 == '>': \n        for i in range(N):\n            for j in range(N):\n                if i != j:\n                    matrix[i,j] = Concordance_degree([at1[j], at1[i]],[at2[i], at2[j]],r)          \n    \n    \n\n    if op1 == '<' and op2 == '<': \n        for i in range(N):\n            for j in range(N):\n                if i != j:\n                    matrix[i,j] = Concordance_degree([at1[j], at1[i]],[at2[j], at2[i]],r)\n"

## Génération de la matrice de d'adjacence. 8 bits
Variante de la fonction précédente, pour le cas où une précision de 8 bits est choisie, des entiers de 8 bits sont utilisés.

In [6]:
# Utilisez des entiers non signés 8 bits au lieu du flotteur python standard qui utilise 64 bits.
def generate_matrix_int(candidate, df, r):
    
    at1 = pd.to_numeric(df.value[candidate[0][0]])
    op1 = candidate[0][1]
    
    at2 = pd.to_numeric(df.value[candidate[1][0]])
    op2 = candidate[1][1]
    
    N = len(at1)     # N est le nombre d'attributs
    matrix = np.zeros((N, N), dtype = np.uint)
    
    if op1 == '>' and op2 == '>': 
        for i in range(N):
            for j in range(N):
                if i != j:
                    matrix[i,j] = (255*Concordance_degree([at1[i], at1[j]],[at2[i], at2[j]],r)) 
                    
    if op1 == '>' and op2 == '<': 
        for i in range(N):
            for j in range(N):
                if i != j:
                    matrix[i,j] = (255*Concordance_degree([at1[i], at1[j]],[at2[j], at2[i]],r))
                    
    return (tuple(candidate), csr(matrix))


##Calcul du support
Renvoie le support de tableau qui lui est passé en argument (qui correspond à la matrice d'adjacence du motif) sous la forme de la somme de tous ses éléments divisée par N(N-1)/2

In [7]:
# Renvoie le support de la matrice
def compute_support(m):
    N = m.shape[1]
    return csr.sum(m)/(N*((N-1)/2))   # Il est également possible d'utiliser directement np.sum.

## calcul du support 8 bits.
Variante de la fonction précédente pour le cas où une précision de 8 bits est choisie.

In [8]:
# Renvoie le support de la matrice
def compute_support_int(m):
    N = m.shape[1]
    return csr.sum(m/255)/(N*((N-1)/2))

## Validation des candidats combinés

Fonction qui retourne la combinaison des itemsets passée en argument si cette combinaison est viable compte tenu des itemsets fréquents de l'itération précédente, évaluant si :

* Ils ont au moins k-2 éléments en commun
* Tous ses sous-ensembles d'itemset incrémentiels de taille k-1 étaient fréquents dans l'itération ou la phase précédente.

La condition si elles ont déjà été générées est évaluée après cette fonction avec un distinct() de la liste obtenue.

In [9]:
def combinedCandidates(itemset_pair, previous_frequent_itemsets):
    #previous_frequent_itemsets = [set(i) for i in previous_frequent_itemsetsk]
    result = ()
    difference = set(itemset_pair[1])-set(itemset_pair[0])
    if(len(difference) == 1):
        result =  itemset_pair[0] + tuple(difference)
        combinations = [x for x in itertools.combinations(result, len(result)-1)]
        if not all(i in previous_frequent_itemsets for i in combinations):
            result = ()

    return result

## Combinaison des matrices
Fonction qui reçoit en argument un itemset combiné et la liste des itemsets fréquents de l'itération précédente, avec leurs matrices respectives. Renvoie l'itemset et la matrice résultant de la combinaison des matrices de deux des itemsets de taille k-1 qui le composent, nous laissant avec le minimum de chacun de ses éléments (norme T de Gödel des matrices).

In [10]:
def ComputeCombinedMatrix(itemset, previous_frequent):
    
    itemset1 = itemset[:-1]
    itemset2 = itemset[1:]
    #itemset2 = itemset[:-2]+itemset[-1]
    
    
    matriz_resultante = csr.minimum(previous_frequent.value[itemset1], previous_frequent.value[itemset2])
    itemset_resultante = itemset1 + tuple(set(itemset2)-set(itemset1))
    
    return (itemset_resultante, matriz_resultante)

## Function principale de l'algorithme.
Renvoie la liste de tous les ensembles d'éléments fréquents dans l'ensemble de données donné, qui dépassent un certain support minimum. Il reçoit en entrée (arguments) le SparkContext, le jeu de données (distribué dans le cluster), le support minimum, un r pour effectuer les appels pour calculer le degré de corrélation entre les attributs. (Utilise une précision de 16 bits pour stocker les degrés d'accord)

In [11]:
def extractFrequentItemsets(sc, distDataset, min_supp, r):
    
    # Premiere etape de l'alogrithme
    candidates = []

    n_att = len(distDataset.value.columns)

    # Nous ajoutons aux candidats tous les itemsets possibles de taille 2 (sauf équivalents)
    for i in range(n_att):
        for j in range(i+1,n_att):
            candidates += [[(distDataset.value.columns[i],'>'),(distDataset.value.columns[j],'>')]]
            candidates += [[(distDataset.value.columns[i],'>'),(distDataset.value.columns[j],'<')]]

    
    # Nous évaluons chacun des candidats possibles de manière distribuée dans le cluster, ne laissant que
    # les itemsets fréquents de taille 2 avec leur tableau associé.
    frequent_itemsets_k = sc.parallelize(candidates).map(lambda x: generate_matrix(x, distDataset, r))\
                                                    .filter(lambda x: compute_support(x[1]) >= min_supp)\
                                                    .collect()
    

    # Seconde phase de l'algorithme

    frequent_itemsets_list = []
    # Nous répétons ce processus jusqu'à ce que la liste des itemsets fréquents générés lors de l'itération précédente soit vide.
    while(frequent_itemsets_k):
        # Nous créons une copie des itemsets fréquents de la phase ou de l'itération précédente et leurs matrices respectives
        # sur chacun des nœuds du cluster.
        previous_frequent_dict = sc.broadcast(dict(frequent_itemsets_k))
        previous_frequent_itemsets = [item for item in previous_frequent_dict.value] # Lista con los itemsets frecuentes anterior
        frequent_itemsets_list += previous_frequent_itemsets
        candidates_combinations = [x for x in itertools.combinations(previous_frequent_itemsets, 2)]

        # Nous évaluons les combinaisons possibles de candidats de manière distribuée dans le cluster, ne laissant que
        # les viables dont le support de la matrice dépasse le support minimum.
        frequent_itemsets_k = sc.parallelize(candidates_combinations)\
                             .map(lambda x: combinedCandidates(x,previous_frequent_itemsets))\
                             .filter(lambda x: x)\
                             .distinct()\
                             .map(lambda x: ComputeCombinedMatrix(x, previous_frequent_dict))\
                             .filter(lambda x: compute_support(x[1]) >= min_supp)\
                             .collect()

    
    # Nous supprimons le contexte actuel.
    sc.stop()
    return frequent_itemsets_list

## Fonction principale identique à ci-dessus mais pour une précision de 8 bits.

In [12]:
def extractFrequentItemsets_int(sc, distDataset, min_supp, r):
    
    # Premiere phase de l'algorithme
    candidates = []

    n_att = len(distDataset.value.columns)

    # Nous ajoutons aux candidats tous les itemsets possibles de taille 2 (sauf équivalents)
    for i in range(n_att):
        for j in range(i+1,n_att):
            candidates += [[(distDataset.value.columns[i],'>'),(distDataset.value.columns[j],'>')]]
            candidates += [[(distDataset.value.columns[i],'>'),(distDataset.value.columns[j],'<')]]

    # Nous évaluons chacun des candidats possibles de manière distribuée dans le cluster, ne laissant que
    # les itemsets fréquents de taille 2 avec leur tableau associé.
    frequent_itemsets_k = sc.parallelize(candidates).map(lambda x: generate_matrix_int(x, distDataset, r))\
                                                    .filter(lambda x: compute_support_int(x[1]) >= min_supp)\
                                                    .collect()

    

    # Seconde phase de l'algorithme

    frequent_itemsets_list = []
    # Nous répétons ce processus jusqu'à ce que la liste des itemsets fréquents générés lors de l'itération précédente soit vide.
    while(frequent_itemsets_k):
        # Nous créons une copie des itemsets fréquents de la phase ou de l'itération précédente et leurs matrices respectives
        # dans chacun des nœuds du cluster.
        previous_frequent_dict = sc.broadcast(dict(frequent_itemsets_k))
        previous_frequent_itemsets = [item for item in previous_frequent_dict.value] # Lista con los itemsets frecuentes anterior
        frequent_itemsets_list += previous_frequent_itemsets
        candidates_combinations = [x for x in itertools.combinations(previous_frequent_itemsets, 2)]

        # Nous évaluons les combinaisons possibles de candidats de manière distribuée dans le cluster, ne laissant que
        # avec les viables dont la matrice dépasse le support minimum.
        frequent_itemsets_k = sc.parallelize(candidates_combinations)\
                             .map(lambda x: combinedCandidates(x,previous_frequent_itemsets))\
                             .filter(lambda x: x)\
                             .distinct()\
                             .map(lambda x: ComputeCombinedMatrix(x, previous_frequent_dict))\
                             .filter(lambda x: compute_support_int(x[1]) >= min_supp)\
                             .collect()

    # Eliminons le contexte actuel.
    sc.stop()
    return frequent_itemsets_list

## On initialise le SparkContext en précisant le nom de l'application et le nombre de cœurs (en cas d'exécution sur une machine locale)

In [26]:
# Initialisation du contexte
def createSparkContext():
    sc_conf = spark.SparkConf()
    sc_conf.setMaster("local[*]")    # Ici, nous pouvons spécifier le nombre de cœurs de processeur que nous voulons utiliser localement
    sc_conf.setAppName("pattern mining")
    #sc_conf.set('spark.executor.instances', '2')
    #sc_conf.set('spark.executor.memory', '2g')
    #sc_conf.set('spark.executor.cores', '1')
    #sc_conf.set('spark.cores.max', '1')
    #sc_conf.set('spark.logConf', True)

    sc = spark.SparkContext(conf=sc_conf)

    return sc 

sc = createSparkContext()

Donnons à Google Collab l'accès aux fichiers du drive, le jeu de données en fait

In [14]:
""""Obligatoire"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [15]:
""""Obligatoire"""
import os
os.chdir("/content/drive/")
%cd MyDrive/TP - DATAMINING - M1 - GRITE
%ls

/content/drive/MyDrive/TP - DATAMINING - M1 - GRITE
'Devoir Datamining - Graduels Patterns Mining.gdoc'   small.csv
 ImplementacionSparkNotebook.ipynb                    winequality-red.csv


## Exécution directe, en attribuant ici les valeurs que l'on veut aux variables

Dans ImplementacionSpark.py (FICHIER SUR LE GITHUB), nous pouvons exécuter et attribuer des paramètres à partir de la ligne de commande

In [27]:
# Execution
r = 0.098

min_supp = 0.3

dataset_file = "winequality-red.csv"

precision = '16bits'

n_att = 12

n_trans = 100

sep = ';'

if(dataset_file == 'pd_speech_features.csv'):
    df = pd.read_csv("pd_speech_features.csv",sep = ",",skiprows = 1)   # Nous l'avons mis de côté car il a une ligne supplémentaire et deux colonnes que nous devons supprimer
    my_data = df.iloc[:n_trans,2:n_att+2]
else:
    # Nous chargeons les données dans une dataframe pandas.
    #df = pd.read_csv("seizure.csv",sep = ",")
    df = pd.read_csv(dataset_file,sep = sep)





# Nous conservons un sous-ensemble total des transactions et des attributs en fonction de la capacité de notre matériel
#my_data = df.iloc[:80,1:-149]
my_data = df.iloc[:50,:]
#my_data = df.iloc[:100,1:-743]


# Normalisation min max des donnees
normalized_data = (my_data - my_data.min())/(my_data.max() - my_data.min())

datadist = sc.broadcast(normalized_data)

if precision == '8bits':
        t1 = time.time()
    
        result = extractFrequentItemsets_int(sc,datadist, min_supp,r)
    
        t2 = time.time()
else:  
        t1 = time.time()
        
        result = extractFrequentItemsets(sc,datadist, min_supp,r)
        
        t2 = time.time()

print("Temps d'execution de l'algorithme: ", t2-t1)
print("\n\nItemsets graduels frequents obtenus avec le support: " + str(min_supp) + "\n\n")
for i in result:
        print(i,"\n")
# Il est nécessaire de réinitialiser le SparkContext si nous voulons exécuter cette cellule plus d'une fois.

Temps d'execution de l'algorithme:  9.186166048049927


Itemsets graduels frequents obtenus avec le support: 0.3


(('fixed acidity', '>'), ('volatile acidity', '<')) 

(('fixed acidity', '>'), ('citric acid', '>')) 

(('fixed acidity', '>'), ('free sulfur dioxide', '>')) 

(('fixed acidity', '>'), ('total sulfur dioxide', '>')) 

(('fixed acidity', '>'), ('density', '>')) 

(('fixed acidity', '>'), ('pH', '<')) 

(('fixed acidity', '>'), ('sulphates', '>')) 

(('fixed acidity', '>'), ('alcohol', '<')) 

(('volatile acidity', '>'), ('citric acid', '<')) 

(('volatile acidity', '>'), ('free sulfur dioxide', '>')) 

(('volatile acidity', '>'), ('free sulfur dioxide', '<')) 

(('volatile acidity', '>'), ('total sulfur dioxide', '>')) 

(('volatile acidity', '>'), ('total sulfur dioxide', '<')) 

(('volatile acidity', '>'), ('density', '>')) 

(('volatile acidity', '>'), ('density', '<')) 

(('volatile acidity', '>'), ('pH', '>')) 

(('volatile acidity', '>'), ('sulphates', '<')) 

(('vola